In [7]:
import requests
import types
from collections import defaultdict

In [44]:
# 一个通用的实时行情获取类
# 行情数据源默认为新浪
# 代码采用新浪代码的表达方式
# 字段名称仿照路透

class Ticker:
    sEngine = 'sina'
    lsSymbols = []
    sSymbols = ''
    dTicker = {}   # defaultdict(dict)
    
    def __init__(self, lsSymbols):
        self.lsSymbols = lsSymbols
        self.sSymbols = ','.join(self.lsSymbols)
    
    def genQuoteDict(self, lsColStr, lsColDef):
        nCols = min(len(lsColStr), len(lsColDef))
        dQuote = {}
        for i in range(0, nCols):
            sColName, sColType = lsColDef[i]
            sColStr = lsColStr[i]
            if sColType == 'float':
                try:
                    dQuote[sColName] = float(sColStr)
                except Exception:
                    dQuote[sColName] = None
            else:
                dQuote[sColName] = sColStr
        #end for
        return dQuote
    #end def
    
    def refreshQuotes(self):
        if self.sEngine == 'sina':
            self.refreshSinaQuotes()
    
    def refreshSinaQuotes(self):
        self.updateQuotesFromSina(self.sSymbols)
    
    def updateQuotesFromSina(self, symbols):
        # Use 'isinstance' to check if obj is an instance of str or any subclass of str.
        # Use 'type(obj) is str' to check if the type of obj is exactly str.
        if not isinstance(symbols, str):
            symbols = ','.join(symbols)
        sUrl = 'http://hq.sinajs.cn/list={}'.format(symbols)
        
        resp = requests.get(sUrl)
        lsLines = resp.text.split('\n')

        
        for sLine in lsLines:
            sLine = sLine.strip()
            if len(sLine) == 0:
                continue
            iBgn  = sLine.find('hq_str_') + 7
            iEnd  = sLine.find('=')
            sCode = sLine[iBgn:iEnd]
            
            iBgn  = sLine.find('"') + 1
            iEnd  = sLine.rfind('"')
            sInfo = sLine[iBgn:iEnd]
            lsCols = sInfo.split(',')
            #nCols = len(lsCols)

            #if lsCols[-1] == '':
            #    lsCols.pop()
            
            lsColDef = []
            if sCode[:2] in ['sh', 'sz']:    # A股
                # 0名称 1今开 2昨收 3现价 4最高 5最低 6买一价 7卖一价 8成交数(股) 9成交额(元)
                # 10(买i申请股数、买i报价)i=1~5  20(卖i申请股数、卖i报价)i=1~5  30日期(yyyy-mm-dd)  31时间(HH:MM:SS)
                # 32状态： "00": "", "01": "临停1H", "02": "停牌", "03": "停牌", "04": "临停",
                #          "05": "停1/2", "07": "暂停", "-1": "无记录", "-2": "未上市", "-3": "退市"
                #lsColNames = ['name', 'open', 'prv_cls', 'now', 'high', 'low', 'bid', 'ask', 'qty', 'amt',
                #              'bsiz1', 'bid1', 'bsiz2', 'bid2', 'bsiz3', 'bid3', 'bsiz4', 'bid4', 'bsiz5', 'bid5',
                #              'asiz1', 'ask1', 'asiz2', 'ask2', 'asiz3', 'ask3', 'asiz4', 'ask4', 'asiz5', 'ask5',
                #              'date', 'time', 'status']
                lsColDef = [('name', 'str'), ('open', 'float'), ('prv_cls', 'float'), ('now', 'float'), ('high', 'float'), 
                            ('low', 'float'), ('bid', 'float'), ('ask', 'float'), ('qty', 'float'), ('amt', 'float'),
                            ('bsiz1', 'float'), ('bid1', 'float'), ('bsiz2', 'float'), ('bid2', 'float'), ('bsiz3', 'float'), 
                            ('bid3', 'float'), ('bsiz4', 'float'), ('bid4', 'float'), ('bsiz5', 'float'), ('bid5', 'float'),
                            ('asiz1', 'float'), ('ask1', 'float'), ('asiz2', 'float'), ('ask2', 'float'), ('asiz3', 'float'), 
                            ('ask3', 'float'), ('asiz4', 'float'), ('ask4', 'float'), ('asiz5', 'float'), ('ask5', 'float'),
                            ('date', 'str'), ('time', 'str'), ('status', 'str')]
            #end if
            
            if lsColDef:
                self.dTicker[sCode] = self.genQuoteDict(lsCols, lsColDef)
            else:
                print('Column definition cannot be found for {}'.format(sLine))
        #end for


In [9]:
dBbCodes = {'sh204001': 1, 'sh204002': 2, 'sh204003': 3, 
            'sh204004': 4, 'sh204007': 7, 'sh204014': 14, 
            'sh204028': 28, 'sh204091': 91, 'sh204182': 182,
            'sz131810': 1, 'sz131811': 2, 'sz131800': 3, 
            'sz131809': 4, 'sz131801': 7, 'sz131802': 14, 
            'sz131803': 28, 'sz131805': 92, 'sz131806': 182}

In [45]:
ticker = Ticker(dBbCodes.keys())

In [46]:
ticker.refreshQuotes()

In [47]:
ticker.dTicker

{'sh204001': {'amt': 971025200000.0,
  'asiz1': 1000.0,
  'asiz2': 1000.0,
  'asiz3': 257000.0,
  'asiz4': 455000.0,
  'asiz5': 8000.0,
  'ask': 4.025,
  'ask1': 4.025,
  'ask2': 4.04,
  'ask3': 4.05,
  'ask4': 4.055,
  'ask5': 4.065,
  'bid': 4.005,
  'bid1': 4.005,
  'bid2': 4.0,
  'bid3': 3.9,
  'bid4': 3.8,
  'bid5': 3.61,
  'bsiz1': 2319000.0,
  'bsiz2': 1606000.0,
  'bsiz3': 200000.0,
  'bsiz4': 1000000.0,
  'bsiz5': 1000000.0,
  'date': '2017-06-26',
  'high': 4.8,
  'low': 3.7,
  'name': 'GC001',
  'now': 4.0,
  'open': 4.0,
  'prv_cls': 3.635,
  'qty': 9710252000.0,
  'status': '00',
  'time': '15:00:00'},
 'sh204002': {'amt': 15087700000.0,
  'asiz1': 2000.0,
  'asiz2': 1000.0,
  'asiz3': 3000.0,
  'asiz4': 44000.0,
  'asiz5': 1000.0,
  'ask': 4.0,
  'ask1': 4.0,
  'ask2': 4.4,
  'ask3': 4.445,
  'ask4': 4.45,
  'ask5': 4.46,
  'bid': 3.72,
  'bid1': 3.72,
  'bid2': 3.7,
  'bid3': 3.55,
  'bid4': 3.51,
  'bid5': 3.4,
  'bsiz1': 5079000.0,
  'bsiz2': 61000.0,
  'bsiz3': 199700

In [19]:
eval('15:00:00')

SyntaxError: invalid syntax (<string>, line 1)

In [20]:
import ast

In [29]:
ast.literal_eval('2017-12-26')

1979

In [ ]:
import ast

# This function cannot be used because this will interpret date strings like '2017-12-26' as an arithmatic equation
def stringToProperType(s):
    try:
        s = ast.literal_eval(s)
    except Exception:
        pass
    return s
#end def

In [ ]:
# This function cannot be used either because the status code '00' will be converted to float 0.0
def stringToProperType(s):
    try:
        s = float(s)
    except Exception:
        pass
    return s
#end def